In [ ]:
import re
import joblib 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [20]:
import pdfplumber
import re

pdf_path = "rewq.pdf"
with pdfplumber.open(pdf_path) as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += page.extract_text()

pattern = r"Total\s*Rs\.?\s*((?:\d\s*)+)"  

match = re.search(pattern, pdf_text)

if match:

    total_claim_amount = int(match.group(1).replace(" ", ""))
    print(f"Total Claim Amount Extracted: Rs {total_claim_amount}")
else:
    print("Total Claim Amount not found in the PDF.")


Total Claim Amount Extracted: Rs 34700


In [21]:
# Regex to capture the "Insured Name" without capturing unwanted text
insured_name_pattern = r"Patient\s*Name[:\-\s]*([A-Za-z\s]+?)(?=\s*(Mobile|Policy|Claim|No))"
insured_name_match = re.search(insured_name_pattern, pdf_text)

if insured_name_match:
    insured_name = insured_name_match.group(1).strip()
    print(f"Patient Name Extracted: {insured_name}")
else:
    print("Patient Name not found in the PDF.")

Patient Name Extracted: Rakshitha Patel


In [12]:
import pdfplumber

# Load the PDF and extract the text
pdf_path = "perote.pdf"
with pdfplumber.open(pdf_path) as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += page.extract_text()

# Save the text to a file for review
with open("pdf_text_debug.txt", "w", encoding="utf-8") as file:
    file.write(pdf_text)

print("Extracted text saved to pdf_text_debug.txt for inspection.")


Extracted text saved to pdf_text_debug.txt for inspection.


In [1]:
import pdf2image
import pytesseract
import pdfplumber
import re
import joblib  
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from PIL import Image

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF using OCR (Tesseract) and pdfplumber."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    
    if not text.strip():  
        images = pdf2image.convert_from_path(pdf_path)
        text = "\n".join([pytesseract.image_to_string(img) for img in images])
    
    return text

def extract_entities(text):
    """Extract hospital name, doctor name, and billing amount using keyword search and regex."""
    hospital_name = "Unknown"
    doctor_name = "Unknown"
    billing_amount = None
    
    lines = text.split("\n")
    for i, line in enumerate(lines):
        if "Name of Insurer" in line:
            hospital_name = lines[i + 1].strip() if i + 1 < len(lines) else "Unknown"
        elif "Doctor Name" in line:
            doctor_name = lines[i + 1].strip() if i + 1 < len(lines) else "Unknown"
    
    
    pattern = r"Total\s*Rs\.?\s*((?:\d\s*)+)"
    match = re.search(pattern, text)
    if match:
        billing_amount = int(match.group(1).replace(" ", ""))
    
    return hospital_name, doctor_name, billing_amount

def train_model(csv_path, model_path):
    """Train an ML model to predict billing amounts with additional features."""
    df = pd.read_csv(csv_path)
    df.dropna(inplace=True)
    
    label_encoders = {}
    for col in ['Doctor', 'Hospital', 'Medical Condition', 'Admission Type', 'Insurance Provider']:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        df[col] = df[col].astype(int)
        label_encoders[col] = le
    
    X = df[['Doctor', 'Hospital', 'Medical Condition', 'Admission Type', 'Insurance Provider']]
    y = df['Billing Amount']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    error = mean_absolute_error(y_test, predictions)
    print(f"Model trained. Mean Absolute Error: {error}")
    
    joblib.dump((model, label_encoders), model_path)
    print(f"Model saved to {model_path}")

def predict_billing(hospital, doctor, medical_condition, admission_type, insurance_provider, model_path):
    """Predict billing amount using a trained ML model with additional features."""
    model, label_encoders = joblib.load(model_path)
    
    def encode_label(value, encoder):
        if value in encoder.classes_:
            return encoder.transform([value])[0]
        else:
            return -1  
    
    features = np.array([
        encode_label(doctor, label_encoders['Doctor']),
        encode_label(hospital, label_encoders['Hospital']),
        encode_label(medical_condition, label_encoders['Medical Condition']),
        encode_label(admission_type, label_encoders['Admission Type']),
        encode_label(insurance_provider, label_encoders['Insurance Provider'])
    ]).reshape(1, -1)
    
    predicted_amount = model.predict(features)
    return predicted_amount[0]

def main(pdf_path, model_path):
    text = extract_text_from_pdf(pdf_path)
    hospital, doctor, actual_amount = extract_entities(text)
    
    medical_condition = "Diabetes"  
    admission_type = "Emergency"  
    insurance_provider = "Medicare"  
    
    predicted_amount = predict_billing(hospital, doctor, medical_condition, admission_type, insurance_provider, model_path)
    
    print(f"Extracted Hospital: {hospital}")
    print(f"Extracted Doctor: {doctor}")
    print(f"Actual Billing Amount: {actual_amount}")
    print(f"Predicted Billing Amount: {predicted_amount}")
    
    if actual_amount:
        difference = abs(actual_amount - predicted_amount)
        print(f"Billing Difference: {difference}")
    else:
        print("Could not extract actual billing amount from PDF.")


In [ ]:
pip install pytesseract pdf2image pillow


Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
train_model("healthcare_dataset.csv", "billing_model.pkl")
main("rewq.pdf", "billing_model.pkl")


Model trained. Mean Absolute Error: 12662.76668211997
Model saved to billing_model.pkl
Extracted Hospital: Paramount
Extracted Doctor: Unknown
Actual Billing Amount: 34700
Predicted Billing Amount: 25729.76015927
Billing Difference: 8970.239840729999


C:\Users\AAYUSH THE GREAT\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
